In [1]:
# ==============================================================================
# CELL 1: 导入所有库并定义最终版的主分析函数
# ==============================================================================
import os
import pandas as pd
import numpy as np

os.environ['OMP_NUM_THREADS'] = '1'

from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from deep_translator import GoogleTranslator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 将所有分析步骤封装到主函数中 ---
def run_analysis(sales_file, unesco_file, reviews_file):
    clear_output(wait=True)
    print(f"--- 正在使用销售文件: '{sales_file}' ---")
    print(f"--- 正在使用UNESCO文件: '{unesco_file}' ---")
    if reviews_file: print(f"--- 正在使用评论文件: '{reviews_file}' ---")
    print("\n" + "="*50 + "\n")

    # ==================== 1. 数据加载与清洗 ====================
    try:
        print("--- 正在加载与清洗数据 ---")
        amazon_df = pd.read_csv(sales_file, dtype={23: str}, on_bad_lines='skip')
        unesco_df = pd.read_csv(unesco_file, encoding="utf-8-sig", on_bad_lines='skip')
        
        # --- 智能重命名逻辑保持不变 ---
        if 'Total Sales' in amazon_df.columns: amazon_df.rename(columns={'Total Sales': 'Amount'}, inplace=True)
        if 'Product' in amazon_df.columns: amazon_df.rename(columns={'Product': 'SKU'}, inplace=True)
        if 'Qty' not in amazon_df.columns and 'Quantity' in amazon_df.columns: amazon_df.rename(columns={'Quantity': 'Qty'}, inplace=True)
        if 'Order ID' not in amazon_df.columns and 'Order_ID' in amazon_df.columns: amazon_df.rename(columns={'Order_ID': 'Order ID'}, inplace=True)
        
        required_cols = ["Amount", "Category", "Date", "Status", "SKU", "Order ID", "Qty"]
        if any(col not in amazon_df.columns for col in required_cols): raise ValueError(f"文件 '{sales_file}' 缺少必需的列。")
        
        amazon_df.dropna(subset=["Amount", "Category", "Date"], inplace=True)
        
        # --- (关键修改) 使用 try-except 来处理日期格式 ---
        try:
            # 优先尝试使用旧文件的高效格式
            amazon_df["Date"] = pd.to_datetime(amazon_df["Date"], format='%m-%d-%y')
            print("日期格式匹配: MM-DD-YY")
        except ValueError:
            # 如果失败，则回退到自动解析，这样可以兼容新文件格式
            print("日期格式不匹配 MM-DD-YY，回退到自动解析...")
            amazon_df["Date"] = pd.to_datetime(amazon_df["Date"], errors='coerce')
            
        amazon_df["Amount"] = pd.to_numeric(amazon_df["Amount"], errors='coerce')
        valid_statuses = ["Shipped", "Shipped - Delivered to Buyer", "Completed", "Pending", "Cancelled"]
        amazon_df = amazon_df[amazon_df["Status"].isin(valid_statuses)]
        amazon_df.dropna(subset=['Date', 'Amount', 'SKU', 'Order ID', 'Qty'], inplace=True)
        
        print("✅ 数据加载和清洗完成！\n")
    except Exception as e:
        print(f"❌ 处理数据时出错: {e}"); return
        
    # (后续的所有分析模块代码保持不变)
    # ...
    # ==================== 2. 过滤与映射 ====================
    print("\n--- 正在进行过滤与映射 ---")
    keywords = ['craft', 'textile', 'embroidery', 'weaving', 'costume', 'dress', 'heritage product', 'handicraft']
    relevant_unesco = unesco_df[unesco_df['Description EN'].str.contains('|'.join(keywords), case=False, na=False)]
    all_categories = amazon_df['Category'].unique()
    non遗_products = amazon_df[amazon_df['Category'].str.contains('|'.join(all_categories), case=False, na=False)]
    print(f"相关非遗数量: {len(relevant_unesco)}")
    print(f"潜在非遗产品订单: {len(non遗_products)}\n")

    # ==================== 3. 时间序列预测 ====================
    print("\n--- 📊 正在生成销售预测图 ---")
    sales_ts = amazon_df.groupby('Date')['Amount'].sum().asfreq('D', fill_value=0)
    model = ARIMA(sales_ts, order=(5, 1, 0)); model_fit = model.fit()
    forecast = model_fit.forecast(steps=30)
    fig_ts = go.Figure()
    fig_ts.add_trace(go.Scatter(x=sales_ts.index, y=sales_ts, name='历史销售额', fill='tozeroy'))
    fig_ts.add_trace(go.Scatter(x=forecast.index, y=forecast, name='预测销售额', fill='tozeroy'))
    fig_ts.update_layout(title='未来30天销售额交互式预测 (ARIMA模型)')
    fig_ts.show()

    # ==================== 4. 类别销售可视化 ====================
    print("\n--- 🛍️ 正在生成品类表现图 ---")
    category_means = non遗_products.groupby('Category')['Amount'].mean().sort_values(ascending=False).reset_index()
    fig_bar = px.bar(category_means, x='Category', y='Amount', color='Category', text_auto='.2f', title='各产品类别平均销售额对比')
    fig_bar.update_layout(width=800, height=500, showlegend=False)
    fig_bar.show()

    # ==================== 5. 商品聚类分析 ====================
    print("\n--- 🔥 正在进行商品聚类分析 ---")
    product_agg_df = amazon_df.groupby('SKU').agg(total_amount=('Amount', 'sum'), total_qty=('Qty', 'sum'), order_count=('Order ID', 'nunique')).reset_index()
    features_to_cluster = ['total_amount', 'total_qty', 'order_count']; features = product_agg_df[features_to_cluster]
    scaler = StandardScaler(); features_scaled = scaler.fit_transform(features)
    kmeans = KMeans(n_clusters=3, n_init=10, random_state=42); product_agg_df.loc[:, 'cluster'] = kmeans.fit_predict(features_scaled)
    cluster_summary = product_agg_df.groupby('cluster')[features_to_cluster].mean().sort_values(by='total_amount', ascending=False)
    hot_product_cluster_id = cluster_summary.index[0]; hot_products = product_agg_df[product_agg_df['cluster'] == hot_product_cluster_id].sort_values(by='total_amount', ascending=False)
    print("\n每个商品簇的特征均值:"); display(cluster_summary)
    print("\n排名前10的热销商品:"); display(hot_products.head(10))
    
    # ==================== 6. (可选) 情感分析 ====================
    if reviews_file:
        print("\n--- 💬 正在进行情感分析 ---")
        try:
            def find_review_column(df):
                priority_cols = ['reviews.text', 'review_text', 'content', 'comment', 'review']
                for p_col in priority_cols:
                    if p_col in df.columns and df[p_col].dropna().astype(str).str.strip().any(): return p_col
                object_cols = df.select_dtypes(include=['object']).columns
                if not object_cols.empty:
                    return max(object_cols, key=lambda col: df[col].dropna().astype(str).str.len().mean())
                return None
            def sentiment_to_rating(sentiment):
                if sentiment >= 0.5: return 5
                elif sentiment >= 0.05: return 4
                elif sentiment > -0.05: return 3
                elif sentiment > -0.5: return 2
                else: return 1
            reviews_df = pd.read_csv(reviews_file)
            review_column_name = find_review_column(reviews_df)
            if review_column_name is None: raise ValueError("未能自动检测到文本列。")
            
            reviews_df.rename(columns={review_column_name: 'review_text'}, inplace=True)
            reviews_df.dropna(subset=['review_text'], inplace=True)
            
            analyzer = SentimentIntensityAnalyzer()
            reviews_df['sentiment'] = reviews_df['review_text'].apply(lambda text: analyzer.polarity_scores(str(text))['compound'])
            
            if 'rating' not in reviews_df.columns:
                reviews_df['rating'] = reviews_df['sentiment'].apply(sentiment_to_rating)
            
            print("\n情感分析结果预览:")
            display(reviews_df.head())
            print("\n高分(>=4星)与低分(<=2星)评论对比:")
            display(reviews_df[reviews_df['rating'] >= 4].head(3))
            display(reviews_df[reviews_df['rating'] <= 2].head(3))
            
        except Exception as e:
            print(f"情感分析失败: {e}")

    # ==================== 7. (可选) 多语言翻译 ====================
    print("\n--- 🌍 正在进行非遗描述翻译 (仅前5条作为演示) ---")
    def translate_text(text, target_lang):
        if not isinstance(text, str) or not text.strip(): return ""
        try: return GoogleTranslator(source='auto', target=target_lang).translate(text)
        except: return text
    target_languages = ['de', 'fr']; unesco_translated_df = unesco_df.head(5).copy()
    for lang in target_languages:
        column_name = f'Description_{lang.upper()}'
        unesco_translated_df[column_name] = unesco_translated_df['Description EN'].apply(lambda x: translate_text(x, lang))
    print("\n翻译完成后的数据预览："); display(unesco_translated_df)
    
    print("\n--- ✨ 分析全部完成 ---")

In [2]:
# ==============================================================================
# CELL 2: 创建并显示交互式界面
# ==============================================================================
import os
from ipywidgets import interactive_output, Dropdown, VBox

# 1. 扫描文件并创建下拉菜单
csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
sales_report_options = [f for f in csv_files if 'amazon' in f.lower() or 'sales' in f.lower()]
unesco_options = [f for f in csv_files if 'ich' in f.lower() or 'unesco' in f.lower()]
reviews_options = [None] + [f for f in csv_files if 'review' in f.lower()]

if not sales_report_options or not unesco_options:
    print("错误：请确保销售文件和UNESCO文件都在当前目录中。")
else:
    # 2. 创建所有控件
    sales_dropdown = Dropdown(options=sales_report_options, description='选择销售文件:')
    unesco_dropdown = Dropdown(options=unesco_options, description='选择UNESCO文件:')
    reviews_dropdown = Dropdown(options=reviews_options, description='选择评论文件 (可选):')

    # 3. 使用 interactive_output 将控件与函数输出分离
    output_area = widgets.Output()

    def on_value_change(change):
        with output_area:
            run_analysis(sales_dropdown.value, unesco_dropdown.value, reviews_dropdown.value)

    # 4. 监听控件值的变化
    sales_dropdown.observe(on_value_change, names='value')
    unesco_dropdown.observe(on_value_change, names='value')
    reviews_dropdown.observe(on_value_change, names='value')

    # 5. 将控件和输出区域一起显示出来
    display(VBox([sales_dropdown, unesco_dropdown, reviews_dropdown, output_area]))
    
    # 6. 首次手动触发运行
    on_value_change(None)